# Tomography Data: Exploratory Data Analysis

This notebook performs EDA on synchrotron X-ray micro-computed tomography
(µCT) and nano-CT data collected at APS tomography beamlines (2-BM, 32-ID).
These beamlines provide high-brilliance, monochromatic X-rays optimized for
fast acquisition of 3D volumetric datasets. Beamline 2-BM supports
full-field µCT at 0.5–20 µm resolution, while 32-ID offers nano-CT
capabilities down to ~50 nm spatial resolution using Fresnel zone plate optics.

We load projection data stored in the HDF5 Data Exchange format, inspect raw
projections and calibration frames (dark/flat fields), apply flat-field
correction, compute sinograms, perform a basic filtered back-projection
reconstruction, and assess image quality metrics such as SNR and resolution.

**Prerequisites**: `pip install h5py numpy matplotlib scipy scikit-image`

In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from scipy import fft as scipy_fft
from scipy.ndimage import median_filter, uniform_filter1d
from skimage.transform import iradon, radon
from skimage.filters import threshold_otsu

plt.rcParams["figure.dpi"] = 120
plt.rcParams["image.cmap"] = "gray"

In [ ]:
# -------------------------------------------------------------------
# Load HDF5 tomography data in Data Exchange format
# The standard groups are:
#   /exchange/data       -- projection images (n_angles x n_rows x n_cols)
#   /exchange/data_dark  -- dark-field images (detector readout noise)
#   /exchange/data_white -- flat-field images (open-beam, no sample)
#   /exchange/theta      -- rotation angles in degrees
# -------------------------------------------------------------------

FILEPATH = "tomo_scan.h5"

with h5py.File(FILEPATH, "r") as f:
    # Load all projection, dark, and flat-field arrays
    projections = f["/exchange/data"][:]
    dark_fields = f["/exchange/data_dark"][:]
    flat_fields = f["/exchange/data_white"][:]
    theta = f["/exchange/theta"][:]

n_angles, n_rows, n_cols = projections.shape
n_darks = dark_fields.shape[0]
n_flats = flat_fields.shape[0]

print(f"Projections:  {projections.shape}  (angles x rows x cols)")
print(f"Dark fields:  {dark_fields.shape}  ({n_darks} frames)")
print(f"Flat fields:  {flat_fields.shape}  ({n_flats} frames)")
print(f"Angle range:  {theta[0]:.2f} to {theta[-1]:.2f} degrees ({n_angles} steps)")
print(f"Pixel dtype:  {projections.dtype}")

## Projection Data Analysis

We visualize representative raw projections at different rotation angles,
the averaged dark-field (detector offset/readout noise), and the averaged
flat-field (open-beam intensity profile without sample in the path).

In [ ]:
# Visualize raw projections at selected angles, plus dark and flat fields
angles_to_show = [0, n_angles // 4, n_angles // 2, 3 * n_angles // 4]

fig, axes = plt.subplots(2, 3, figsize=(16, 10))

# Raw projections at four angles
for i, idx in enumerate(angles_to_show[:4]):
    ax = axes.ravel()[i]
    im = ax.imshow(projections[idx], cmap="gray")
    ax.set_title(f"Projection @ {theta[idx]:.1f}\u00b0", fontsize=10)
    plt.colorbar(im, ax=ax, fraction=0.046)

# Averaged dark field -- detector offset/readout noise
dark_avg = np.mean(dark_fields, axis=0)
im4 = axes[1, 0].imshow(dark_avg, cmap="gray")
axes[1, 0].set_title(f"Avg Dark Field (mean={dark_avg.mean():.1f})", fontsize=10)
plt.colorbar(im4, ax=axes[1, 0], fraction=0.046)

# Averaged flat field -- open beam profile
flat_avg = np.mean(flat_fields, axis=0)
im5 = axes[1, 1].imshow(flat_avg, cmap="gray")
axes[1, 1].set_title(f"Avg Flat Field (mean={flat_avg.mean():.1f})", fontsize=10)
plt.colorbar(im5, ax=axes[1, 1], fraction=0.046)

# Effective beam profile: flat - dark
beam_profile = flat_avg - dark_avg
im6 = axes[1, 2].imshow(beam_profile, cmap="viridis")
axes[1, 2].set_title("Flat \u2212 Dark (beam profile)", fontsize=10)
plt.colorbar(im6, ax=axes[1, 2], fraction=0.046)

plt.suptitle("Raw Projections, Dark and Flat Fields", fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Extract a sinogram from the raw projection stack
# A sinogram is a single detector row viewed across all rotation angles;
# each row of the sinogram corresponds to one angular projection.

sino_row = n_rows // 2  # middle row of the detector
sinogram_raw = projections[:, sino_row, :]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Display sinogram as 2D image
axes[0].imshow(sinogram_raw, aspect="auto", cmap="gray",
               extent=[0, n_cols, theta[-1], theta[0]])
axes[0].set_xlabel("Detector Column (pixels)")
axes[0].set_ylabel("Rotation Angle (degrees)")
axes[0].set_title(f"Raw Sinogram (row {sino_row})")

# Intensity profile at the center column across all angles
center_col = n_cols // 2
axes[1].plot(theta, sinogram_raw[:, center_col], "b-", lw=0.8)
axes[1].set_xlabel("Rotation Angle (degrees)")
axes[1].set_ylabel("Raw Intensity")
axes[1].set_title(f"Sinogram Trace at Column {center_col}")
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Sinogram shape: {sinogram_raw.shape}")
print(f"Intensity range: [{sinogram_raw.min():.1f}, {sinogram_raw.max():.1f}]")

## Preprocessing

Standard tomography preprocessing includes:

1. **Flat-field correction**: Normalizes projections by the incident beam profile
   and removes detector-specific artifacts. The corrected transmission is
   `(proj - dark) / (flat - dark)`.
2. **Negative logarithm**: Converts transmission data to line integrals of the
   linear attenuation coefficient via the Beer-Lambert law: `-ln(I/I0)`.
3. **Ring artifact removal**: Addresses concentric ring artifacts in reconstructed
   slices caused by defective or miscalibrated detector pixels, which appear as
   vertical stripes in the sinogram domain.

In [ ]:
# Flat-field correction
# normalized = (projection - dark) / (flat - dark)
# This yields the sample transmission T = I/I0 at each pixel

dark_avg = np.mean(dark_fields, axis=0).astype(np.float32)
flat_avg = np.mean(flat_fields, axis=0).astype(np.float32)

# Avoid division by zero where flat == dark
denom = flat_avg - dark_avg
denom[denom < 1.0] = 1.0

# Normalize all projections
proj_norm = np.zeros_like(projections, dtype=np.float32)
for i in range(n_angles):
    proj_norm[i] = (projections[i].astype(np.float32) - dark_avg) / denom

# Clip to valid transmission range [epsilon, 1]
proj_norm = np.clip(proj_norm, 1e-6, 1.0)

# Visualize before/after for one projection
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].imshow(projections[0], cmap="gray")
axes[0].set_title("Raw Projection (0\u00b0)")

axes[1].imshow(proj_norm[0], cmap="gray", vmin=0, vmax=1)
axes[1].set_title("Flat-Field Corrected (0\u00b0)")

plt.tight_layout()
plt.show()

print(f"Normalized projection range: [{proj_norm.min():.4f}, {proj_norm.max():.4f}]")
print(f"Mean transmission: {proj_norm.mean():.4f}")

In [ ]:
# Negative log transform: convert transmission to attenuation line integrals
# Beer-Lambert law: -ln(I/I0) = integral of mu(x,y) along the ray path

proj_log = -np.log(proj_norm)

# Extract the corrected sinogram at the same row
sinogram = proj_log[:, sino_row, :]

# --- Ring artifact removal concept ---
# Vertical stripes in the sinogram correspond to concentric ring artifacts
# in the reconstruction. A simple approach: compute the column-wise mean,
# smooth it, and subtract the difference (high-frequency residual) to
# reduce fixed-pattern detector non-uniformities.

def remove_ring_artifacts(sinogram, kernel_size=51):
    """Simple ring removal via column-mean high-pass filtering."""
    col_mean = np.mean(sinogram, axis=0)
    col_mean_smooth = uniform_filter1d(col_mean, size=kernel_size)
    correction = col_mean - col_mean_smooth
    return sinogram - correction[np.newaxis, :]

sinogram_clean = remove_ring_artifacts(sinogram)

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

axes[0].imshow(sinogram, aspect="auto", cmap="gray")
axes[0].set_title("Corrected Sinogram (-log)")
axes[0].set_xlabel("Detector Column")
axes[0].set_ylabel("Angle Index")

axes[1].imshow(sinogram_clean, aspect="auto", cmap="gray")
axes[1].set_title("After Ring Removal")
axes[1].set_xlabel("Detector Column")

# Column-mean comparison before/after ring removal
axes[2].plot(np.mean(sinogram, axis=0), "r-", lw=0.8, label="Before")
axes[2].plot(np.mean(sinogram_clean, axis=0), "b-", lw=0.8, label="After")
axes[2].set_xlabel("Column")
axes[2].set_ylabel("Mean Attenuation")
axes[2].set_title("Column-Mean Profile")
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Reconstruction Preview

We reconstruct a single axial slice using filtered back-projection (FBP)
via `skimage.transform.iradon`. This provides a quick preview to verify
data quality, center-of-rotation alignment, and preprocessing effectiveness
before committing to a full 3D reconstruction.

In [ ]:
# Simple FBP reconstruction of a single sinogram slice
# iradon expects sinogram shape (n_detector_pixels, n_angles)

sinogram_fbp = sinogram_clean.T  # transpose to (n_cols, n_angles)

# Reconstruct using Ram-Lak (ramp) filter -- standard for absorption CT
recon_slice = iradon(sinogram_fbp, theta=theta, filter_name="ramp", circle=True)

# Also try Shepp-Logan filter for comparison (slightly smoother)
recon_slice_sl = iradon(sinogram_fbp, theta=theta, filter_name="shepp-logan", circle=True)

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Input sinogram
axes[0].imshow(sinogram_clean, aspect="auto", cmap="gray")
axes[0].set_title(f"Input Sinogram (row {sino_row})")
axes[0].set_xlabel("Detector Column")
axes[0].set_ylabel("Angle Index")

# Ramp filter reconstruction
im1 = axes[1].imshow(recon_slice, cmap="gray")
axes[1].set_title("FBP Reconstruction (Ramp)")
plt.colorbar(im1, ax=axes[1], fraction=0.046, label="\u03bc (1/pixel)")

# Shepp-Logan filter reconstruction
im2 = axes[2].imshow(recon_slice_sl, cmap="gray")
axes[2].set_title("FBP Reconstruction (Shepp-Logan)")
plt.colorbar(im2, ax=axes[2], fraction=0.046, label="\u03bc (1/pixel)")

plt.suptitle(f"Filtered Back-Projection Reconstruction (slice row {sino_row})", fontsize=13)
plt.tight_layout()
plt.show()

print(f"Reconstructed slice shape: {recon_slice.shape}")
print(f"Value range (Ramp):        [{recon_slice.min():.4f}, {recon_slice.max():.4f}]")
print(f"Value range (Shepp-Logan): [{recon_slice_sl.min():.4f}, {recon_slice_sl.max():.4f}]")

## Statistical Analysis

We examine the distribution of attenuation values in the reconstructed slice
to identify material phases and estimate signal-to-noise ratio (SNR) and
contrast-to-noise ratio (CNR) between distinct regions.

In [ ]:
# Histogram of reconstructed attenuation values and SNR estimation

# Create circular mask (reconstruction is valid inside the inscribed circle)
ny, nx = recon_slice.shape
yy, xx = np.mgrid[:ny, :nx]
center = np.array([ny / 2, nx / 2])
radius = min(ny, nx) / 2 * 0.95
circle_mask = ((yy - center[0])**2 + (xx - center[1])**2) < radius**2

valid_values = recon_slice[circle_mask]

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Histogram of all valid attenuation values
axes[0].hist(valid_values, bins=200, color="steelblue", edgecolor="none", log=True)
axes[0].set_xlabel("Attenuation Coefficient (1/pixel)")
axes[0].set_ylabel("Count (log)")
axes[0].set_title("Attenuation Value Histogram")
axes[0].axvline(0, color="red", ls="--", lw=1, label="air (\u03bc=0)")

# Use Otsu thresholding to separate air from material
try:
    thresh = threshold_otsu(valid_values)
    axes[0].axvline(thresh, color="orange", ls="--", lw=1, label=f"Otsu={thresh:.4f}")
    axes[0].legend(fontsize=8)
except ValueError:
    thresh = np.median(valid_values)

# SNR estimation: compare material region to background (air)
air_mask = circle_mask & (recon_slice < thresh * 0.3)
material_mask = circle_mask & (recon_slice > thresh)

if air_mask.any() and material_mask.any():
    air_mean = recon_slice[air_mask].mean()
    air_std = recon_slice[air_mask].std()
    mat_mean = recon_slice[material_mask].mean()
    mat_std = recon_slice[material_mask].std()
    snr = abs(mat_mean - air_mean) / max(air_std, 1e-10)
    cnr = abs(mat_mean - air_mean) / np.sqrt(air_std**2 + mat_std**2)

    axes[1].bar(["Air", "Material"], [air_mean, mat_mean],
                yerr=[air_std, mat_std], capsize=5, color=["skyblue", "coral"])
    axes[1].set_ylabel("Mean \u03bc")
    axes[1].set_title(f"SNR = {snr:.1f}, CNR = {cnr:.1f}")

    # Segmentation overlay visualization
    overlay = np.zeros((*recon_slice.shape, 3))
    norm_img = (recon_slice - recon_slice.min()) / (recon_slice.max() - recon_slice.min() + 1e-10)
    overlay[..., 0] = norm_img
    overlay[..., 1] = norm_img
    overlay[..., 2] = norm_img
    overlay[material_mask, 0] = 1.0  # highlight material in red
    overlay[material_mask, 1] *= 0.3
    overlay[material_mask, 2] *= 0.3
    axes[2].imshow(overlay)
    axes[2].set_title("Material Segmentation Overlay")
else:
    axes[1].text(0.5, 0.5, "Insufficient data", ha="center", transform=axes[1].transAxes)
    axes[2].text(0.5, 0.5, "Insufficient data", ha="center", transform=axes[2].transAxes)

plt.tight_layout()
plt.show()

print(f"Air region:      mean={air_mean:.5f}, std={air_std:.5f}")
print(f"Material region: mean={mat_mean:.5f}, std={mat_std:.5f}")
print(f"SNR = {snr:.1f},  CNR = {cnr:.1f}")

In [ ]:
# Resolution estimation concepts:
# Modulation Transfer Function (MTF) and Edge Spread Function (ESF)
#
# Spatial resolution of a CT reconstruction can be estimated by:
# 1. Finding a sharp edge (material-air boundary) in the reconstructed slice
# 2. Computing the Edge Spread Function (ESF) across that edge
# 3. Differentiating ESF -> Line Spread Function (LSF)
# 4. Fourier transforming LSF -> Modulation Transfer Function (MTF)
# 5. The spatial frequency at which MTF drops to 10% gives the resolution limit

def estimate_resolution_from_edge(recon, row, col_start, col_end):
    """Estimate spatial resolution from an edge profile in the reconstruction."""
    # Extract edge spread function (ESF)
    esf = recon[row, col_start:col_end].astype(np.float64)
    
    # Line spread function (LSF) = derivative of ESF
    lsf = np.gradient(esf)
    
    # Modulation transfer function (MTF) = |FFT(LSF)|
    mtf = np.abs(np.fft.rfft(lsf))
    mtf = mtf / mtf[0]  # normalize to DC component
    
    # Spatial frequency axis (cycles per pixel)
    freq = np.fft.rfftfreq(len(lsf))
    
    return esf, lsf, mtf, freq

# Select an edge region from the reconstruction
# In practice, identify a sharp material-air boundary
edge_row = ny // 2
col_start = nx // 4
col_end = 3 * nx // 4

esf, lsf, mtf, freq = estimate_resolution_from_edge(
    recon_slice, edge_row, col_start, col_end
)

fig, axes = plt.subplots(1, 3, figsize=(16, 4))

axes[0].plot(esf, "b-", lw=1)
axes[0].set_xlabel("Position (pixels)")
axes[0].set_ylabel("Intensity")
axes[0].set_title("Edge Spread Function (ESF)")
axes[0].grid(True, alpha=0.3)

axes[1].plot(lsf, "r-", lw=1)
axes[1].set_xlabel("Position (pixels)")
axes[1].set_ylabel("d(Intensity)/dx")
axes[1].set_title("Line Spread Function (LSF)")
axes[1].grid(True, alpha=0.3)

axes[2].plot(freq, mtf, "g-", lw=1)
axes[2].axhline(0.1, color="red", ls="--", lw=1, label="MTF = 10%")
axes[2].set_xlabel("Spatial Frequency (cycles/pixel)")
axes[2].set_ylabel("MTF")
axes[2].set_title("Modulation Transfer Function (MTF)")
axes[2].set_ylim(0, 1.1)
axes[2].legend(fontsize=8)
axes[2].grid(True, alpha=0.3)

# Find 10% MTF cutoff frequency
above_10 = np.where(mtf > 0.1)[0]
if len(above_10) > 0:
    cutoff_idx = above_10[-1]
    cutoff_freq = freq[cutoff_idx]
    resolution_pixels = 1.0 / max(cutoff_freq, 1e-6)
    print(f"MTF 10% cutoff frequency: {cutoff_freq:.4f} cycles/pixel")
    print(f"Estimated resolution:     {resolution_pixels:.1f} pixels")
else:
    print("Could not determine MTF cutoff (no clear edge found in selected region)")

plt.tight_layout()
plt.show()

## Data Scale Considerations

Synchrotron tomography datasets can be extremely large. Understanding the data
volume is critical for planning storage, I/O strategy, and computational resources.
A typical µCT scan at APS 2-BM produces 1500–3000 projections at 2048×2048 pixels,
each stored as 16-bit or 32-bit values.

In [ ]:
# Calculate dataset sizes for typical synchrotron tomography configurations

configs = [
    {"name": "Standard \u00b5CT (2-BM)",     "n_proj": 1500, "rows": 2048, "cols": 2048, "bytes_per_pixel": 4},
    {"name": "High-res \u00b5CT (2-BM)",     "n_proj": 3000, "rows": 2048, "cols": 2048, "bytes_per_pixel": 4},
    {"name": "Nano-CT (32-ID)",          "n_proj": 1200, "rows": 2048, "cols": 2048, "bytes_per_pixel": 2},
    {"name": "Fast tomo (2-BM)",         "n_proj": 900,  "rows": 2048, "cols": 2048, "bytes_per_pixel": 2},
    {"name": "4D tomo (time-resolved)",  "n_proj": 900,  "rows": 2048, "cols": 2048, "bytes_per_pixel": 2},
]

print(f"{'Configuration':<30s} {'Projections':>12s} {'Size (pixels)':>14s} "
      f"{'Raw (GB)':>10s} {'Recon (GB)':>10s}")
print("-" * 80)

for cfg in configs:
    # Raw data: n_proj * rows * cols * bytes_per_pixel
    raw_bytes = cfg["n_proj"] * cfg["rows"] * cfg["cols"] * cfg["bytes_per_pixel"]
    # Reconstruction volume: cols x cols x rows voxels (float32 output)
    recon_bytes = cfg["cols"] * cfg["cols"] * cfg["rows"] * 4
    raw_gb = raw_bytes / (1024**3)
    recon_gb = recon_bytes / (1024**3)
    print(f"{cfg['name']:<30s} {cfg['n_proj']:>12d} {cfg['rows']:>6d}x{cfg['cols']:<6d} "
          f"{raw_gb:>10.2f} {recon_gb:>10.2f}")

# Current dataset summary
current_raw = projections.nbytes / (1024**3)
current_dark = dark_fields.nbytes / (1024**3)
current_flat = flat_fields.nbytes / (1024**3)

print(f"\n--- Current Dataset ---")
print(f"Projections: {current_raw:.3f} GB  ({projections.shape}, {projections.dtype})")
print(f"Dark fields: {current_dark:.3f} GB  ({dark_fields.shape})")
print(f"Flat fields: {current_flat:.3f} GB  ({flat_fields.shape})")
print(f"Total raw:   {current_raw + current_dark + current_flat:.3f} GB")
print(f"\nNote: A single day of beam time at APS 2-BM can generate 1-10 TB of raw data.")

## EDA Summary

After running this notebook, you should have assessed:

1. **Data structure** -- Verified projection dimensions, angle range, and HDF5 Data Exchange layout
2. **Raw data quality** -- Inspected projections at multiple angles, dark/flat fields, and beam profile
3. **Sinogram extraction** -- Confirmed sinogram construction from the projection stack
4. **Flat-field correction** -- Normalized projections to transmission values using `(proj - dark) / (flat - dark)`
5. **Preprocessing** -- Applied negative log transform and ring artifact removal
6. **Reconstruction** -- Verified FBP reconstruction quality with Ramp and Shepp-Logan filters
7. **Statistical analysis** -- Examined attenuation histograms, SNR, and CNR
8. **Resolution** -- Estimated spatial resolution via MTF/ESF analysis
9. **Data scale** -- Assessed storage and computational requirements for typical beamline configurations

**Next steps**: Full 3D reconstruction (using TomoPy, ASTRA, or TomocuPy),
center-of-rotation optimization, phase retrieval (for phase-contrast imaging at 32-ID),
segmentation, and quantitative morphological analysis.